In [1]:
#!/usr/bin/env python
# coding: utf-8

# --- NOTES -------------------------------------------------------------------
# 1. Update the datasets, dataList
# -----------------------------------------------------------------------------

import os
import re
import sys
import json
import time
import pyspark
from copy import deepcopy
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.functions import udf, unix_timestamp, col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType, TimestampType
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline 
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.types import StringType
from pyspark.sql.functions import array
import pyspark.sql.functions as f
import csv
from pyspark.sql.functions import *
from pyspark.ml.feature import * 
from pyspark.sql.types import *
from nltk.corpus import stopwords

In [2]:
# -----------------------------------------------------------------------------
# --- Function Definitions Begin ----------------------------------------------

def write_keyword_list_to_txt(lst, dest_filename):
    with open(dest_filename, 'w') as f:
        wr = csv.writer(f, quoting=csv.QUOTE_ALL)
        wr.writerow(lst)

# --- Function Definitions End ------------------------------------------------
# -----------------------------------------------------------------------------

In [3]:
# -----------------------------------------------------------------------------
# --- MAIN --------------------------------------------------------------------

if __name__ == "__main__":
    # Setting spark context and 
    sc = SparkContext()
    spark = SparkSession \
        .builder \
        .appName("project_task1") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    sqlContext = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)


In [4]:

    # Current user path
    env_var = os.environ
    this_user = env_var['USER']

    # Input & output directories
    #inputDirectory = "/user/hm74/NYCOpenData/"#sys.argv[1]
    inputDirectory = "/home/ted/school/big_data/project/big_data_course_project/task2/raw_data/"
    inputFileClusters = "/home/ted/school/big_data/project/big_data_course_project/task2/resources/filename_clusters.json"
    #outputDirectory = "/user/" + this_user + "/project/task1/"#sys.argv[2]

    # Output JSON Semantic Schema
    semanticSchema = {
        "semantic_type": "",
        "count": 0
    }

    # Importing cluster3 format it and put it into a list
    raw_data = sc.textFile("cluster3.txt")
    raw_list = raw_data.flatMap(lambda x: x.split(",")).collect()
    raw_list = [re.sub("\[|\]|\'|\'|" "", "", item)for item in raw_list]
    raw_list = [re.sub(" " "", "", item)for item in raw_list]
    
    # Iteration over dataframes begins bu using dataframe file names
    processCount = 1

    #df_new = df_split_words.withColumn("word", array(df_split_words["word"]))

    # Create schema for raw data before reading into df 
    customSchema = StructType([
        #StructField("My_array", ArrayType(
        #StructType([StructField("val", StringType())]))   
        StructField("val", StringType(), True),
        StructField("count", IntegerType(), True)])    
 

In [ ]:
# area of study filelist (aos)
aos_list = sc.textFile("areas_of_study")
aos_list = aos_list.flatMap(lambda x: x.split(",")).collect()
aos_list = [x.strip('"') for x in aos_list]
aos_list = [re.sub("\[|\]|\'|\'|" "", "", item)for item in aos_list]
aos_list = [re.sub(" " "", "", item)for item in aos_list]

# generate keyword list for areas of study
aos_keyword_list = []
aos_current_list = []
for filename in aos_list: # first. change to map later
    aos_df = sqlContext.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").schema(customSchema).load(inputDirectory + filename)
    aos_df_clean = aos_df.select("val", f.regexp_replace(f.col("val"), "[\$#&,]", "").alias("clean_word"))

    # split words in each row and create new df with one word per row, and count
    aos_df_split_words = aos_df_clean.withColumn('word', f.explode(f.split(f.col('clean_word'), ' ')))\
        .groupBy('word')\
        .count()\
        .sort('count', ascending=False)\
        .filter("word != ''")
        
    aos_current_list = [row['word'] for row in aos_df_split_words.take(20)]
    aos_keyword_list = list(set(aos_keyword_list).union(set(aos_current_list)))
    
# remove stopwords
filtered_aos_keywords = [word for word in aos_keyword_list if word not in stopwords.words('english')]

# write to file
write_keyword_list_to_txt(filtered_aos_keywords, 'area_of_study_keywords')


In [ ]:
filtered_aos_keywords

In [ ]:
# city agency filelist (ca)
ca_list = sc.textFile("agency_filelist")
ca_list = ca_list.flatMap(lambda x: x.split(",")).collect()
ca_list = [x.strip('"') for x in ca_list]
ca_list = [re.sub("\[|\]|\'|\'|" "", "", item)for item in ca_list]
ca_list = [re.sub(" " "", "", item)for item in ca_list]

# generate keyword list for city agencies
ca_current_list = []
ca_keyword_list = []
for filename in ca_list: # first. change to map later
    ca_df = sqlContext.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").schema(customSchema).load(inputDirectory + filename)
    ca_df_clean = ca_df.select("val", f.regexp_replace(f.col("val"), "[\-$#&,]", "").alias("clean_word"))

    # split words in each row and create new df with one word per row, and count
    ca_df_split_words = ca_df_clean.withColumn('word', f.explode(f.split(f.lower(f.col('clean_word')), ' ')))\
        .groupBy('word')\
        .count()\
        .sort('count', ascending=False)\
        .filter("word != ''")

    #ca_df_new = ca_df_split_words.withColumn("word", array(ca_df_split_words["word"]))
       
    ca_current_list = [row['word'] for row in ca_df_split_words.take(30)]
    ca_keyword_list = list(set(ca_keyword_list).union(set(ca_current_list)))

# remove stopwords (sw list is lowercase)
filtered_ca_keywords = [word for word in ca_keyword_list if word not in stopwords.words('english')]

# back to uppercase to match 
filtered_ca_keywords = [x.upper() for x in filtered_ca_keywords]

write_keyword_list_to_txt(filtered_ca_keywords, 'city_agency_keywords')
  

In [ ]:
# parks and playgrounds (pp) filelist
pp_list = sc.textFile("park_playground_filelist")
pp_list = pp_list.flatMap(lambda x: x.split(",")).collect()
pp_list = [x.strip('"') for x in pp_list]
pp_list = [re.sub("\[|\]|\'|\'|" "", "", item)for item in pp_list]
pp_list = [re.sub(" " "", "", item)for item in pp_list]

# generate keyword list for city agencies
pp_current_list = []
pp_keyword_list = []
for filename in pp_list: # first. change to map later
    pp_df = sqlContext.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").schema(customSchema).load(inputDirectory + filename)
    pp_df_clean = pp_df.select("val", f.regexp_replace(f.col("val"), "[\$#&,]", "").alias("clean_word"))
        
    # split words in each row and create new df with one word per row, and count
    pp_df_split_words = pp_df_clean.withColumn('word', f.explode(f.split(f.lower(f.col('val')), ' ')))\
        .groupBy('word')\
        .count()\
        .sort('count', ascending=False)\
        .filter("word != ''")
    #pp_df_new = pp_df_split_words.withColumn("word", array(pp_df_split_words["word"]))
    
        
    pp_current_list = [row['word'] for row in pp_df_split_words.take(40)]
    pp_keyword_list = list(set(pp_keyword_list).union(set(pp_current_list)))

# remove stopwords and other words which may result in a missclassification
# TODO: clean this up...
pp_keyword_list.remove('-')
pp_keyword_list.remove('school')
pp_keyword_list.remove('academy')
pp_keyword_list.remove('charter')
pp_keyword_list.remove('high')
pp_keyword_list.remove('jhs')
pp_keyword_list.remove('middle')
pp_keyword_list.remove('secondary')
pp_keyword_list.remove('senior')
pp_keyword_list.remove('h')
pp_keyword_list.remove('j')
pp_keyword_list.remove('e')
#pp_keyword_list.remove('c')
#pp_keyword_list.remove('f')
#pp_keyword_list.remove('w')
#pp_keyword_list.remove('b')
#pp_keyword_list.remove('r')
#pp_keyword_list.remove('l')
pp_keyword_list.remove('st')
#pp_keyword_list.remove('st.')
filtered_pp_keywords = [word for word in pp_keyword_list if word not in stopwords.words('english')]

# back to uppercase to match original dataset
filtered_pp_keywords = [x.upper() for x in filtered_pp_keywords]

write_keyword_list_to_txt(filtered_pp_keywords, 'park_playground_keywords')

In [ ]:
# NEIGHBORHOODS

# neighborhood filelist (nh)
nh_list = sc.textFile("neighborhood_filelist")
nh_list = nh_list.flatMap(lambda x: x.split(",")).collect()
nh_list = [x.strip('"') for x in nh_list]
nh_list = [re.sub("\[|\]|\'|\'|" "", "", item)for item in nh_list]
nh_list = [re.sub(" " "", "", item)for item in nh_list]

# generate keyword list for city agencies
nh_current_list = []
nh_keyword_list = []
for filename in nh_list: # first. change to map later
    nh_df = sqlContext.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").schema(customSchema).load(inputDirectory + filename)
    nh_df_clean = nh_df.select("val", f.regexp_replace(f.col("val"), "[\-$#&,]", "").alias("clean_word"))

    # split words in each row and create new df with one word per row, and count
    nh_df_split_words = nh_df_clean.withColumn('word', f.explode(f.split(f.lower(f.col('clean_word')), ' ')))\
        .groupBy('word')\
        .count()\
        .sort('count', ascending=False)\
        .filter("word != ''")

    #ca_df_new = ca_df_split_words.withColumn("word", array(ca_df_split_words["word"]))
       
    nh_current_list = [row['word'] for row in nh_df_split_words.take(20)]
    nh_keyword_list = list(set(nh_keyword_list).union(set(nh_current_list)))

# remove stopwords (sw list is lowercase)
nh_keyword_list.remove('st')
nh_keyword_list.remove('avenue')
#nh_keyword_list.remove('')
filtered_nh_keywords = [word for word in nh_keyword_list if word not in stopwords.words('english')]

# back to uppercase to match 
filtered_nh_keywords = [x.upper() for x in filtered_nh_keywords]

write_keyword_list_to_txt(filtered_nh_keywords, 'neighborhood_keywords')

In [16]:
# LOCATION TYPE

# location type filelist (lt)
lt_list = sc.textFile("location_type_filelist")
lt_list = lt_list.flatMap(lambda x: x.split(",")).collect()
lt_list = [x.strip('"') for x in lt_list]
lt_list = [re.sub("\[|\]|\'|\'|" "", "", item)for item in lt_list]
lt_list = [re.sub(" " "", "", item)for item in lt_list]

# generate keyword list for city agencies
lt_current_list = []
lt_keyword_list = []
for filename in lt_list: # first. change to map later
    print(filename)
    lt_df = sqlContext.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").schema(customSchema).load(inputDirectory + filename)
    lt_df_clean = lt_df.select("val", f.regexp_replace(f.col("val"), "[\-$#&,]", "").alias("clean_word"))

    # split words in each row and create new df with one word per row, and count
    lt_df_split_words = lt_df_clean.withColumn('word', f.explode(f.split(f.lower(f.col('clean_word')), ' ')))\
        .groupBy('word')\
        .count()\
        .sort('count', ascending=False)\
        .filter("word != ''")

    #ca_df_new = ca_df_split_words.withColumn("word", array(ca_df_split_words["word"]))
       
    lt_current_list = [row['word'] for row in lt_df_split_words.take(100)]
    lt_keyword_list = list(set(lt_keyword_list).union(set(lt_current_list)))

# remove stopwords (sw list is lowercase)
#lt_keyword_list.remove('street')
lt_keyword_list.remove('nyc')
filtered_lt_keywords = [word for word in lt_keyword_list if word not in stopwords.words('english')]

# back to uppercase to match 
filtered_lt_keywords = [x.upper() for x in filtered_lt_keywords]

write_keyword_list_to_txt(filtered_lt_keywords, 'location_type_keywords')

5uac-w243.PREM_TYP_DESC.txt.gz
qgea-i56i.PREM_TYP_DESC.txt.gz


In [23]:
# SCHOOL NAME

# school name type filelist (sn)
sn_list = sc.textFile("school_name_type_filelist")
sn_list = sn_list.flatMap(lambda x: x.split(",")).collect()
sn_list = [x.strip('"') for x in sn_list]
sn_list = [re.sub("\[|\]|\'|\'|" "", "", item)for item in sn_list]
sn_list = [re.sub(" " "", "", item)for item in sn_list]

# generate keyword list for city agencies
sn_current_list = []
sn_keyword_list = []
for filename in sn_list: # first. change to map later
    print(filename)
    sn_df = sqlContext.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").schema(customSchema).load(inputDirectory + filename)
    sn_df_clean = sn_df.select("val", f.regexp_replace(f.col("val"), "[\-$#&,]", "").alias("clean_word"))

    # split words in each row and create new df with one word per row, and count
    sn_df_split_words = sn_df_clean.withColumn('word', f.explode(f.split(f.lower(f.col('clean_word')), ' ')))\
        .groupBy('word')\
        .count()\
        .sort('count', ascending=False)\
        .filter("word != ''")

    #ca_df_new = ca_df_split_words.withColumn("word", array(ca_df_split_words["word"]))
       
    sn_current_list = [row['word'] for row in sn_df_split_words.take(20)]
    sn_keyword_list = list(set(sn_keyword_list).union(set(sn_current_list)))

# remove stopwords (sw list is lowercase)
#lt_keyword_list.remove('street')
#sn_keyword_list.remove('nyc')
filtered_sn_keywords = [word for word in sn_keyword_list if word not in stopwords.words('english')]

# back to uppercase to match 
filtered_sn_keywords = [x.upper() for x in filtered_sn_keywords]

write_keyword_list_to_txt(filtered_sn_keywords, 'school_name_type_keywords')

4y63-yw9e.SCHOOL_NAME.txt.gz
gk83-aa6y.SCHOOL_NAME.txt.gz
xne4-4v8f.SCHOOL.txt.gz
sqcr-6mww.School_Name.txt.gz
5e7x-8jy6.School_Name.txt.gz
3rfa-3xsf.School_Name.txt.gz
wks3-66bn.School_Name.txt.gz
hy4q-igkk.School_Name.txt.gz
x3kb-2vbv.School_Name.txt.gz
imfa-v5pv.School_Name.txt.gz
jtus-srrj.School_Name.txt.gz
jzt2-2f7h.School_Name.txt.gz


In [29]:
# SCHOOL SUBJECT

# school subject type filelist (ss)
ss_list = sc.textFile("school_subject_type_filelist")
ss_list = ss_list.flatMap(lambda x: x.split(",")).collect()
ss_list = [x.strip('"') for x in ss_list]
ss_list = [re.sub("\[|\]|\'|\'|" "", "", item)for item in ss_list]
ss_list = [re.sub(" " "", "", item)for item in ss_list]

# generate keyword list for city agencies
ss_current_list = []
ss_keyword_list = []
for filename in ss_list: # first. change to map later
    print(filename)
    ss_df = sqlContext.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").schema(customSchema).load(inputDirectory + filename)
    ss_df_clean = ss_df.select("val", f.regexp_replace(f.col("val"), "[\-$#&,]", "").alias("clean_word"))

    # split words in each row and create new df with one word per row, and count
    ss_df_split_words = ss_df_clean.withColumn('word', f.explode(f.split(f.lower(f.col('clean_word')), ' ')))\
        .groupBy('word')\
        .count()\
        .sort('count', ascending=False)\
        .filter("word != ''")

    #ca_df_new = ca_df_split_words.withColumn("word", array(ca_df_split_words["word"]))
       
    ss_current_list = [row['word'] for row in ss_df_split_words.take(50)]
    ss_keyword_list = list(set(ss_keyword_list).union(set(ss_current_list)))

# remove stopwords (sw list is lowercase)
#lt_keyword_list.remove('street')
ss_keyword_list.remove('11')
ss_keyword_list.remove('10')
ss_keyword_list.remove('b')
ss_keyword_list.remove('9')
ss_keyword_list.remove('12')
filtered_ss_keywords = [word for word in ss_keyword_list if word not in stopwords.words('english')]

# back to uppercase to match 
filtered_ss_keywords = [x.upper() for x in filtered_ss_keywords]

write_keyword_list_to_txt(filtered_ss_keywords, 'school_subject_type_keywords')

d3ge-anaz.CORE_COURSE__MS_CORE_and_9_12_ONLY_.txt.gz
6wcu-cfa3.CORE_COURSE__MS_CORE_and_9_12_ONLY_.txt.gz
ajgi-hpq9.CORE_SUBJECT___MS_CORE_and__09_12_ONLY_.txt.gz
ub9e-s7ai.CORE_SUBJECT___MS_CORE_and__09_12_ONLY_.txt.gz
pgtq-ht5f.CORE_SUBJECT___MS_CORE_and__9_12_ONLY_.txt.gz
ytjm-yias.CORE_SUBJECT___MS_CORE_and__09_12_ONLY_.txt.gz
5nz7-hh6t.CORE_SUBJECT___MS_CORE_and__09_12_ONLY_.txt.gz
gez6-674h.CORE_SUBJECT___MS_CORE_and__9_12_ONLY_.txt.gz
f7qh-bcr5.CORE_SUBJECT___MS_CORE_and__9_12_ONLY_.txt.gz
7yds-6i8e.CORE_SUBJECT__MS_CORE_and_9_12_ONLY_.txt.gz
kz72-dump.CORE_SUBJECT___MS_CORE_and__09_12_ONLY_.txt.gz
sybh-s59s.CORE_SUBJECT___MS_CORE_and__9_12_ONLY_.txt.gz
6ypq-ih9a.CORE_SUBJECT___MS_CORE_and__09_12_ONLY_.txt.gz
3aka-ggej.CORE_SUBJECT___MS_CORE_and__09_12_ONLY_.txt.gz
i8ys-e4pm.CORE_COURSE_9_12_ONLY_.txt.gz
8i43-kna8.CORE_SUBJECT.txt.gz


In [30]:
filtered_ss_keywords

['GEOMETRY',
 'HISTORY',
 'ECONOMICS',
 'SECTIONS',
 'SOCIAL',
 'US',
 'STUDIES',
 'CHEMISTRY',
 'EARTH',
 'TEAM',
 'MATH',
 'GLOBAL',
 'GOVERNMENT',
 'LIVING',
 'SCIENCE',
 'MATCHED',
 'ENGLISH',
 'PHYSICS',
 'TEACHING',
 'ENVIRONMENT',
 'ALGEBRA',
 'ASSUMED']

In [ ]:

# other option: do this instead of above if you want to mine frequent itemsets later
# (probably useful for strings with at least a few words)
#pp_df_split_words = pp_df.withColumn('word', f.explode(f.split(f.col('val'), ' ')))\
#    .select('word')
#pp_df.show()
#pp_df_new = pp_df.select(col("val"), split(col("val"), " \s*").cast(ArrayType(StringType())).alias("word"))
pp_df_new = pp_df.withColumn("word", array(pp_df["val"]))


fpGrowth = FPGrowth(itemsCol="word", minSupport=0.00, minConfidence=0.0)
model = fpGrowth.fit(pp_df_new)
model.freqItemsets.show()
#pp_df_new.show()
#model.associationRules.show()
#model.transform(pp_df_new).show()

In [ ]:

# other option: do this instead of above if you want to mine frequent itemsets later
# (probably useful for strings with at least a few words)
#df_split_words = df.withColumn('word', f.explode(f.split(f.col('val'), ' ')))\
#    .select('word')
#df_new = df_split_words.withColumn("word", array(df_split_words["word"]))
#fpGrowth = FPGrowth(itemsCol="word", minSupport=0.1, minConfidence=0.3)
#model = fpGrowth.fit(df_new)
#model.freqItemsets.show()
#df_new.show()
